# Generator

Ferramenta que pega no conteudo das noticias (em formato json) e gera posts.
Para isso são executados os seguintes passos:

   1) Obtenção do core do texto:
      -  T5 
      - Allegation sentence

   2) Reforço de emoção: 
      - Emojis
      - Hahstags
      - Caps lock
      - interjection

   3) Reforço de personalidade
      Avalia o processo mental predominante, recorrendo ao menTFIDF;
      Reforça o processo mental predominante.



In [19]:
## Importes necessarios

import os

import json
import pandas as pd
import random
from men_tf_idf import MenTFIDF


from nltk.corpus import wordnet as wn

import pandas as pd
import matplotlib.pyplot as plt
from nrclex import NRCLex
import nltk
import spacy
import json
nltk.download('punkt')


nlp = spacy.load('en_core_web_md')



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mariabarbosa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
'''
verificar se um post (em formato de string) tem um formato válido de um microblog:
     1) 0 < número de carateres ≥ 280 
'''


def verify_length(post):
    content = ""

    for i in post:
        if type(post[i]) == str:
            content = content + " " + post[i] 
        else:
            content = content + " " + ' '.join(post[i]) 
    if len(content)>280:
        return False
    return True

In [21]:
from TextCore import T5 

def allegation_sentence (data):
    claim = "It's " + data['evaluation'] + ' that '+ data['allegation']
    return claim


def post_core (data, core_model):
    
    if core_model == "t5":
        core = T5.T5_summarization_model(' '.join(data['postText']))
    else:
        core = allegation_sentence (data)
    return core

In [22]:
from emotions import hashtags_generetor
from emotions import emojis
from emotions import interjections
from emotions import caps_lock



def add_emotion(data,core, emoji , hashtag , capslock  , intejections ):
    post = {}
    post['url'] = data['url']
    ## Não adiciona mais conteudo por isso não é necessario verificar tamanho
    if capslock: 
        post['core']  = caps_lock.capslock(core)
    else:
        post['core'] = core

    post['emojis'] = []
    if emoji and verify_length(post): 
        post['emojis'].append(emojis.emojis_sellection(data['evaluation']))

    post['hashtags'] = []
    if hashtag and verify_length(post):
        hashtag_list = hashtags_generetor.generateHashtags(data['postText'])
        for h in hashtag_list:
            post['hashtags'].append('#'+h)
    
    post['interjections'] = [] 
    if intejections and verify_length(post): 
        post['interjections'].append(interjections.interjections(data['evaluation']))

    if emoji and verify_length(post): 
        #adicionar mais emojis se ainda
        seed = random.randint(0,4)
        while verify_length(post) and seed < 4:
            post['emojis'].append(emojis.emojis_sellection(data['evaluation']))
            seed = seed + 1
    ## Post fica com ordem alaetoria.
   # core_with_emotion = random.shuffle(post) 

    return post

In [23]:
def Core_generator (data,core_model='t5', emoji = 1, hashtag = 1, caps_lock =1 , intejections = 1):
  '''
  data -> Conteudo do json file
  '''
  
  ## Obter o core do post
  core = post_core (data, core_model)
  
  ## dar emoção ao core
  core_with_emotion = add_emotion(data, core, emoji , hashtag , caps_lock  , intejections )

  ## dar personalidade
  final_post = core_with_emotion  

  return final_post  

        
        

In [24]:
def creat_post_structure(post_data):

    lista_post = [post_data['core']]

    for h in  post_data['hashtags']:
        lista_post.append(h)

    for e in  post_data['emojis']:
        lista_post.append(e)

    for i in  post_data['interjections']:
        lista_post.append(i)

    random.shuffle(lista_post) 

    lista_post.append(post_data['url'])
    post = ' '.join(lista_post)
    return post 


In [ ]:
'''
##### Read the json files with de news
for file in os.listdir("../../extractors/Snopes/exemplo"):
    with open("../../extractors/Snopes/exemplo/" + file) as f:
        data = json.load(f)
        f.close()
'''



In [25]:
import pandas as pd

df = pd.read_csv('MentaLex/MentaLex_synonyms.csv')
            

In [26]:
'''
Funções que permitem reforçar o processo mental
Isto feito atraves de substituições de palavras que não pertencem ao leixco desse traço mental, por sinonimos que pertençam 
'''

def Schizoid (sentence):
    words_sentence = []
    new_sentence = []
    for word in sentence.split(' '):
        words_sentence.append(word)

    for w in words_sentence: 
        if w not in list_shizoid:
            ''' 
            Analisar os sinonimos de w e ver se algum deles esta na lista. Se estiver substituir
            '''

            syn = synonyms(w)
            f = 0
            for s in syn:
                if s in list_shizoid:
                    f = 1
                    new_sentence.append(s)
                    break
            if f==0:
                 new_sentence.append(w)
                
        else:
            new_sentence.append(w)

            
    return new_sentence

def Paranoid (sentence):
    words_sentence = []
    new_sentence = []
    for word in sentence.split(' '):
        words_sentence.append(word)

    for w in words_sentence: 
        if w not in list_paranoid:
            ''' 
            Analisar os sinonimos de w e ver se algum deles esta na lista. Se estiver substituir
            '''

            syn = synonyms(w)
            f = 0
            for s in syn:
                if s in list_paranoid:
                    f = 1
                    new_sentence.append(s)
                    break
            if f==0:
                 new_sentence.append(w)
                
        else:
            new_sentence.append(w)

            
    return new_sentence



def Neurotic (sentence):
    words_sentence = []
    new_sentence = []
    for word in sentence.split(' '):
        words_sentence.append(word)

    for w in words_sentence: 
        if w not in list_neurotic:
            ''' 
            Analisar os sinonimos de w e ver se algum deles esta na lista. Se estiver substituir
            Verificar que a palavra n esteja nas que não se deve incluir
            '''

            syn = synonyms(w)
            f = 0
            for s in syn:
                if s in list_neurotic:
                    f = 1
                    new_sentence.append(s)
                    break
            if f==0:
                 new_sentence.append(w)
                
        else:
            new_sentence.append(w)

            
    return new_sentence




list_paranoid = list(df[df['classification'] == 'Paranoid']['word'])
list_neurotic = list(df[df['classification'] == 'Neuroticism']['word'])
list_shizoid = list(df[df['classification'] == 'Schizoid']['word'])


def synonyms (word):

    synonyms = []

    for syn in wn.synsets(word):
            for l in syn.lemmas():
                doc = nlp(l.name())
                s = [ str(token) for token in doc ] 
                synonyms = synonyms + s
                    
    # print(set(synonyms_traits))

    return synonyms



In [31]:

''''
Algoritmo geral
'''

def postGenerator (filePath, model='allegation_Sentence'):
    with open(filePath) as file:
        data = json.load(file)
        file.close()

    #Gerar Text Core
    p = Core_generator(data, model)
    # Adicionar a emoção
    post_with_emotion = creat_post_structure(p)

    #verificar processo mental predominante pelo calculo do valor TFIDF
    menTFIDF  = MenTFIDF() 
    menTFIDF.set_text(post_with_emotion)
    frq = menTFIDF.em_frequencies

    ## ordenar por valor de frequencia
    mental_tfidf_frequencies = {k: v for k, v in sorted(frq.items(), key=lambda item: item[1], reverse=True)}
    
        # extrair traço mental predominante
    mental_process = list(mental_tfidf_frequencies.keys())[0]

    #De acordo com traço mental predominante reforçar esse traço
    if mental_process == "Neurocitism":

        final_post  = Neurotic (post_with_emotion)
    elif mental_process == "Paranoid":
        final_post = Paranoid (post_with_emotion)
    else:
        final_post = Schizoid(post_with_emotion)


    return (' '.join(final_post))  




In [33]:

'''
for file in os.listdir("../../extractors/Snopes/news")[2:3]:
        with open("../../extractors/Snopes/news/" + file) as f:
            data = json.load(f)
            f.close()
            for a in ['allegation_Sentence', 't5']:
                p = generator(data, a)
                post_with_emotion = creat_post_structure(p)
                pers= personality (post_with_emotion)
               
'''

file = (os.listdir("../../extractors/Snopes/news")[2])
print(file)
postGenerator ("../../extractors/Snopes/news/" + file)



snopes_extraction_1663673769.393436.json


"😂 👅 #john It's true that A VIDEO of a yearn SNAKE “standing up” and lift its head high into the strain follow real. #black oh 👭 #snake 🏈 #cobra │ https://www.snopes.com/fact-check/snake-standing-up-video/?utm_term=Autofeed&utm_medium=Social&utm_source=Twitter#Echobox=1663406617"

In [ ]:
''''
i = 0
lista = []
for file in os.listdir("../../extractors/Snopes/news"):
        with open("../../extractors/Snopes/news/" + file) as f:
            data = json.load(f)
            f.close()
            for a in ['allegation_Sentence', 't5']:
                p = generator(data, a)
                post_with_emotion = creat_post_structure(p)
                pers= personality (post_with_emotion)
                lista.append([i,a,post_with_emotion, pers])
                i = i +1

df = pd.DataFrame(lista, columns=['id', 'aproach', 'post', 'personality'])
print(df)



##########################

df.to_csv('post_news.csv', index = False)
import pandas as pd
df = pd.read_csv('post_news.csv')

df['personality'][10]
'''    


In [1]:
from nrclex import NRCLex

def emotionsNRClex(text):
  
    emotion = NRCLex(text)
    emotions_list = emotion.affect_frequencies
    return emotions_list


